In [0]:
from tensorflow.keras.datasets.cifar10 import load_data

In [3]:
(x_train, y_train), (x_test, y_test) = load_data()

170500096/170498071 [==============================] - 9s 0us/step


In [4]:
print(x_train.shape , y_train.shape)
print(x_test.shape, y_test.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [16]:
import tensorflow as tf
import numpy as np

X = tf.placeholder(tf.float32, [None, 32*32*3])
Y = tf.placeholder(tf.float32, [None, 10])
print(X.shape, Y.shape)

W1 = tf.Variable(tf.random_normal([3072,512], stddev=0.01))
B1 = tf.Variable(tf.random_normal([512], stddev=0.01))
L1 = tf.nn.relu(tf.matmul(X,W1)+B1)
print(W1.shape, L1.shape)

W2 = tf.Variable(tf.random_normal([512,512], stddev = 0.01))
B2 = tf.Variable(tf.random_normal([512], stddev=0.01))
L2 = tf.nn.relu(tf.matmul(L1,W2)+B2)
print(W2.shape, L2.shape)

W3 = tf.Variable(tf.random_normal([512,256], stddev = 0.01))
B3 = tf.Variable(tf.random_normal([256], stddev=0.01))
L3 = tf.nn.relu(tf.matmul(L2,W3)+B3)
print(W3.shape, L3.shape)

W4 = tf.Variable(tf.random_normal([256,256], stddev = 0.01))
B4 = tf.Variable(tf.random_normal([256], stddev = 0.01))
L4 = tf.nn.relu(tf.matmul(L3,W4)+B4)
print(W4.shape, L4.shape)

W5 = tf.Variable(tf.random_normal([256,10], stddev = 0.01))
B5 = tf.Variable(tf.random_normal([10], stddev=0.01))
model = tf.matmul(L4, W5)+B5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

init = tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)
  
  num_epoch = 100 
  batch_size = 2500
  total_batch = int(len(x_train)/batch_size)
  
  for epoch in range(num_epoch):
    total_cost = 0
    
    for i in range(total_batch):
      if i == (total_batch-1):
        order = list(range(len(x_train)))
        import random
        random.shuffle(order)
        x_train = x_train[order]
        y_train = y_train[order]
        
      batch_xs = x_train[i*batch_size:(i+1)*batch_size].reshape(batch_size, -1)
      batch_ys = np.eye(10)[y_train[i*batch_size:(i+1)*batch_size]].reshape(batch_size,-1)
      #print(batch_xs.shape)
      #print(batch_ys.shape)
      
      _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})  # keep_prob is 80 percent here
      total_cost += cost_val
      #print(cost_val)
      #print(_)

    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))   
  
  print('최적화 완료!')
  
  is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
  accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
  #print('정확도:', sess.run(accuracy,
  #                        feed_dict={X: x_test.reshape(len(x_test), -1),
  #                                  Y: np.eye(10)[y_test].reshape(len(y_test),-1)}))
  print("Accuracy:" , accuracy.eval({X : x_test.reshape(len(x_test),-1),
                                     Y : np.eye(10)[y_test].reshape(len(y_test),-1)}))

(?, 3072) (?, 10)
(3072, 512) (?, 512)
(512, 512) (?, 512)
(512, 256) (?, 256)
(256, 256) (?, 256)
Epoch: 0001 Avg. cost = 2.208
Epoch: 0002 Avg. cost = 2.022
Epoch: 0003 Avg. cost = 1.906
Epoch: 0004 Avg. cost = 1.829
Epoch: 0005 Avg. cost = 1.763
Epoch: 0006 Avg. cost = 1.735
Epoch: 0007 Avg. cost = 1.685
Epoch: 0008 Avg. cost = 1.677
Epoch: 0009 Avg. cost = 1.625
Epoch: 0010 Avg. cost = 1.595
Epoch: 0011 Avg. cost = 1.581
Epoch: 0012 Avg. cost = 1.547
Epoch: 0013 Avg. cost = 1.519
Epoch: 0014 Avg. cost = 1.509
Epoch: 0015 Avg. cost = 1.478
Epoch: 0016 Avg. cost = 1.432
Epoch: 0017 Avg. cost = 1.439
Epoch: 0018 Avg. cost = 1.418
Epoch: 0019 Avg. cost = 1.396
Epoch: 0020 Avg. cost = 1.368
Epoch: 0021 Avg. cost = 1.377
Epoch: 0022 Avg. cost = 1.361
Epoch: 0023 Avg. cost = 1.337
Epoch: 0024 Avg. cost = 1.299
Epoch: 0025 Avg. cost = 1.327
Epoch: 0026 Avg. cost = 1.283
Epoch: 0027 Avg. cost = 1.256
Epoch: 0028 Avg. cost = 1.220
Epoch: 0029 Avg. cost = 1.222
Epoch: 0030 Avg. cost = 1.227
E

In [25]:
is_training = True

X = tf.placeholder(tf.float32, [None, 32*32*3])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
print(X.shape, Y.shape)

W1 = tf.Variable(tf.random_normal([3072,512], stddev=0.01))
B1 = tf.Variable(tf.random_normal([512], stddev=0.01))
L1 = tf.matmul(X,W1)+B1
L1_batch = tf.layers.batch_normalization(L1, training=is_training)
L1_act = tf.nn.relu(L1_batch)
L1_act = tf.nn.dropout(L1_act, keep_prob)
print(W1.shape, L1.shape)

W2 = tf.Variable(tf.random_normal([512,512], stddev = 0.01))
B2 = tf.Variable(tf.random_normal([512], stddev=0.01))
L2 = tf.matmul(L1_act,W2)+B2
L2_batch = tf.layers.batch_normalization(L2, training=is_training)
L2_act = tf.nn.relu(L2_batch)
L2_act = tf.nn.dropout(L2_act, keep_prob)
print(W2.shape, L2.shape)

W3 = tf.Variable(tf.random_normal([512,256], stddev = 0.01))
B3 = tf.Variable(tf.random_normal([256], stddev=0.01))
L3 = tf.matmul(L2_act,W3)+B3
L3_batch = tf.layers.batch_normalization(L3, training=is_training)
L3_act = tf.nn.relu(L3_batch)
L3_act = tf.nn.dropout(L3_act, keep_prob)
print(W3.shape, L3.shape)

W4 = tf.Variable(tf.random_normal([256,256], stddev = 0.01))
B4 = tf.Variable(tf.random_normal([256], stddev = 0.01))
L4 = tf.matmul(L3_act,W4)+B4
L4_batch = tf.layers.batch_normalization(L4, training=is_training)
L4_act = tf.nn.relu(L4_batch)
L4_act = tf.nn.dropout(L4_act, keep_prob)
print(W4.shape, L4.shape)

W5 = tf.Variable(tf.random_normal([256,10], stddev = 0.01))
B5 = tf.Variable(tf.random_normal([10], stddev=0.01))
model = tf.matmul(L4, W5)+B5

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

init = tf.global_variables_initializer()
with tf.Session() as sess:
  sess.run(init)
  
  num_epoch = 100 
  batch_size = 2500
  total_batch = int(len(x_train)/batch_size)
  
  for epoch in range(num_epoch):
    total_cost = 0
    
    for i in range(total_batch):
      if i == (total_batch-1):
        order = list(range(len(x_train)))
        import random
        random.shuffle(order)
        x_train = x_train[order]
        y_train = y_train[order]
        
      batch_xs = x_train[i*batch_size:(i+1)*batch_size].reshape(batch_size, -1)
      batch_ys = np.eye(10)[y_train[i*batch_size:(i+1)*batch_size]].reshape(batch_size,-1)
      #print(batch_xs.shape)
      #print(batch_ys.shape)
      
      _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.8})  # keep_prob is 80 percent here
      total_cost += cost_val
      #print(cost_val)
      #print(_)

    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))   
  
  print('최적화 완료!')
  
  is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
  accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
  #print('정확도:', sess.run(accuracy,
  #                        feed_dict={X: x_test.reshape(len(x_test), -1),
  #                                  Y: np.eye(10)[y_test].reshape(len(y_test),-1)}))
  print("Accuracy:" , accuracy.eval({X : x_test.reshape(len(x_test),-1),
                                     Y : np.eye(10)[y_test].reshape(len(y_test),-1),
                                     keep_prob: 1}))

(?, 3072) (?, 10)
(3072, 512) (?, 512)
(512, 512) (?, 512)
(512, 256) (?, 256)
(256, 256) (?, 256)
Epoch: 0001 Avg. cost = 1.916
Epoch: 0002 Avg. cost = 1.579
Epoch: 0003 Avg. cost = 1.453
Epoch: 0004 Avg. cost = 1.369
Epoch: 0005 Avg. cost = 1.302
Epoch: 0006 Avg. cost = 1.248
Epoch: 0007 Avg. cost = 1.208
Epoch: 0008 Avg. cost = 1.164
Epoch: 0009 Avg. cost = 1.140
Epoch: 0010 Avg. cost = 1.097
Epoch: 0011 Avg. cost = 1.057
Epoch: 0012 Avg. cost = 1.025
Epoch: 0013 Avg. cost = 0.994
Epoch: 0014 Avg. cost = 0.972
Epoch: 0015 Avg. cost = 0.936
Epoch: 0016 Avg. cost = 0.911
Epoch: 0017 Avg. cost = 0.887
Epoch: 0018 Avg. cost = 0.863
Epoch: 0019 Avg. cost = 0.841
Epoch: 0020 Avg. cost = 0.820
Epoch: 0021 Avg. cost = 0.799
Epoch: 0022 Avg. cost = 0.761
Epoch: 0023 Avg. cost = 0.755
Epoch: 0024 Avg. cost = 0.734
Epoch: 0025 Avg. cost = 0.721
Epoch: 0026 Avg. cost = 0.694
Epoch: 0027 Avg. cost = 0.679
Epoch: 0028 Avg. cost = 0.663
Epoch: 0029 Avg. cost = 0.635
Epoch: 0030 Avg. cost = 0.634
E